In [31]:

!pip install configparser
!pip install pandas
import tweepy
import pandas as pd
import numpy as np
import requests
import os
import re
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
data_load = pd.read_csv("/content/drive/MyDrive/fyp/fyp2/tweets16m_cleaned.csv.csv")
display(data_load)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date,date.1,tweet,lang
0,2017-01-01,2017-01-01 10:30:10+00,2814 BTC],en
1,2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00. High $975.00. L...,en
2,2017-01-01,2017-01-01 10:30:06+00,8BitCoin 8BIT $0.011464 (0.86%) 0.00001195 BTC...,en
3,2017-01-01,2017-01-01 10:23:44+00,Bitcoin last trade\n\n $976.81\n $939.00\n\nSe...,en
4,2017-01-01,2017-01-01 10:30:20+00,$977.38 bitfinex;\n$974.98 GDAX;\n$975.00 bits...,en
...,...,...,...,...
11299370,2019-11-23,2019-11-23 05:40:39+00,"Bitcoin price 7238.65641481, DOWN -4.74 percen...",en
11299371,2019-11-23,2019-11-23 05:40:02+00,A $XMR is worth 0.00696045 BTC,en
11299372,2019-11-23,2019-11-23 05:40:03+00,"Top 5 cryptocurrencies \n03\n $7,234.439\n $15...",en
11299373,2019-11-23,2019-11-23 05:39:53+00,"Arbistar 2.0, llamada de éxito 20/11, bots for...",en


In [33]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
df = data_load.copy()
df.replace("", np.nan, inplace=True) # replace empty string with np.nan
df.dropna(subset = ["tweet"], inplace=True) # drop nan values

In [35]:
from nltk.tokenize import word_tokenize, RegexpTokenizer

import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()
sentence = "quake"

new_words = {
    'bull': 2,
    'bear': -1.2,
    'bullish': 2,
    'bearish': -1.2,
    'dump': -1.5

}

analyzer.lexicon.update(new_words)
s = analyzer.polarity_scores(sentence)
print(s)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [36]:
neg = []
neu = []
pos = []
compound = []
display(df)
#df_t = df.copy()
for tweet in df['tweet']:
  s = analyzer.polarity_scores(tweet)
  neg.append(s['neg'])
  neu.append(s['neu'])
  pos.append(s['pos'])
  compound.append(s['compound'])

df['neg_intensity'] = neg
df['neu_intensity'] = neu
df['pos_intensity'] = pos
df['compound_intensity'] = compound


,date,date.1,tweet,lang
0,2017-01-01,2017-01-01 10:30:10+00,2814 BTC],en
1,2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00. High $975.00. L...,en
2,2017-01-01,2017-01-01 10:30:06+00,8BitCoin 8BIT $0.011464 (0.86%) 0.00001195 BTC...,en
3,2017-01-01,2017-01-01 10:23:44+00,Bitcoin last trade\n\n $976.81\n $939.00\n\nSe...,en
4,2017-01-01,2017-01-01 10:30:20+00,$977.38 bitfinex;\n$974.98 GDAX;\n$975.00 bits...,en
...,...,...,...,...
11299370,2019-11-23,2019-11-23 05:40:39+00,"Bitcoin price 7238.65641481, DOWN -4.74 percen...",en
11299371,2019-11-23,2019-11-23 05:40:02+00,A $XMR is worth 0.00696045 BTC,en
11299372,2019-11-23,2019-11-23 05:40:03+00,"Top 5 cryptocurrencies \n03\n $7,234.439\n $15...",en
11299373,2019-11-23,2019-11-23 05:39:53+00,"Arbistar 2.0, llamada de éxito 20/11, bots for...",en


In [37]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# init the sentiment analyzer
sia = SentimentIntensityAnalyzer()
score_df = []
#display(df.insert('score'))

for sentence in df['tweet']:
    score = sia.polarity_scores(sentence)["compound"]
    #print(f'The sentiment value of the sentence :"{sentence}" is : {score}')
    score_df.append(score)
    #print(score)

df_score = pd.DataFrame(score_df,columns=['score'] )
display(df_score)


,score
0,0.0000
1,-0.0516
2,0.0000
3,0.0000
4,0.0000
...,...
10822068,0.3400
10822069,0.2263
10822070,0.2023
10822071,0.0000


In [38]:
#df = df_clean
display(df_score)
display(df)
df_t = df.copy()
df_t['vader_score'] = df_score
display(df_t)
#df_score.to_csv('/content/drive/MyDrive/fyp/fyp2/scores/textblob_score.csv')
df_t.to_csv('/content/drive/MyDrive/fyp/fyp2/scores/vader_score.csv')

,score
0,0.0000
1,-0.0516
2,0.0000
3,0.0000
4,0.0000
...,...
10822068,0.3400
10822069,0.2263
10822070,0.2023
10822071,0.0000


,date,date.1,tweet,lang,neg_intensity,neu_intensity,pos_intensity,compound_intensity
0,2017-01-01,2017-01-01 10:30:10+00,2814 BTC],en,0.000,1.000,0.000,0.0000
1,2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00. High $975.00. L...,en,0.131,0.750,0.119,-0.0516
2,2017-01-01,2017-01-01 10:30:06+00,8BitCoin 8BIT $0.011464 (0.86%) 0.00001195 BTC...,en,0.000,1.000,0.000,0.0000
3,2017-01-01,2017-01-01 10:23:44+00,Bitcoin last trade\n\n $976.81\n $939.00\n\nSe...,en,0.000,1.000,0.000,0.0000
4,2017-01-01,2017-01-01 10:30:20+00,$977.38 bitfinex;\n$974.98 GDAX;\n$975.00 bits...,en,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
11299370,2019-11-23,2019-11-23 05:40:39+00,"Bitcoin price 7238.65641481, DOWN -4.74 percen...",en,0.000,0.888,0.112,0.3400
11299371,2019-11-23,2019-11-23 05:40:02+00,A $XMR is worth 0.00696045 BTC,en,0.000,0.725,0.275,0.2263
11299372,2019-11-23,2019-11-23 05:40:03+00,"Top 5 cryptocurrencies \n03\n $7,234.439\n $15...",en,0.000,0.878,0.122,0.2023
11299373,2019-11-23,2019-11-23 05:39:53+00,"Arbistar 2.0, llamada de éxito 20/11, bots for...",en,0.000,1.000,0.000,0.0000


,date,date.1,tweet,lang,neg_intensity,neu_intensity,pos_intensity,compound_intensity,vader_score
0,2017-01-01,2017-01-01 10:30:10+00,2814 BTC],en,0.000,1.000,0.000,0.0000,0.0000
1,2017-01-01,2017-01-01 10:30:04+00,One Bitcoin now worth $975.00. High $975.00. L...,en,0.131,0.750,0.119,-0.0516,-0.0516
2,2017-01-01,2017-01-01 10:30:06+00,8BitCoin 8BIT $0.011464 (0.86%) 0.00001195 BTC...,en,0.000,1.000,0.000,0.0000,0.0000
3,2017-01-01,2017-01-01 10:23:44+00,Bitcoin last trade\n\n $976.81\n $939.00\n\nSe...,en,0.000,1.000,0.000,0.0000,0.0000
4,2017-01-01,2017-01-01 10:30:20+00,$977.38 bitfinex;\n$974.98 GDAX;\n$975.00 bits...,en,0.000,1.000,0.000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
11299370,2019-11-23,2019-11-23 05:40:39+00,"Bitcoin price 7238.65641481, DOWN -4.74 percen...",en,0.000,0.888,0.112,0.3400,NaN
11299371,2019-11-23,2019-11-23 05:40:02+00,A $XMR is worth 0.00696045 BTC,en,0.000,0.725,0.275,0.2263,NaN
11299372,2019-11-23,2019-11-23 05:40:03+00,"Top 5 cryptocurrencies \n03\n $7,234.439\n $15...",en,0.000,0.878,0.122,0.2023,NaN
11299373,2019-11-23,2019-11-23 05:39:53+00,"Arbistar 2.0, llamada de éxito 20/11, bots for...",en,0.000,1.000,0.000,0.0000,NaN
